## libraries and globals

In [7]:
import dill as pickle
import csv
import re
import numpy as np
import networkx as nx
# #
from scipy.stats import fisher_exact
from scipy.sparse import csr_matrix
from datetime import datetime
from collections import Counter, defaultdict
from unidecode import unidecode
from editdistance import eval as ed
from itertools import chain, combinations
from multiprocessing import Pool
import json

In [2]:
# BITEXT FUNCTIONS
good_pos = {'NOUN','ADJ','VERB'}
excluded_lemmas = {'be', 'other', 'have', 'let', 'one', 'lot', 'same', 'such', 't', 's'}

## Liu et al reimplementation

In [3]:
## fragment extraction

def get_fragments(w, min_len=2, max_len=7, yield_self=True):
    """
    takes a string *w* and extracts all substrings of minimal length *min_len* and maximal length *max_len*
    """
    if yield_self and len(w) > max_len: yield w
    for i in range(len(w)):
        for j in range(i+min_len, min(i+max_len+1,len(w)+1)):
            if (not (i == 0 or j == len(w))) or (j-i) >= min_len+1:
                yield w[i:j]

def get_all_fragments(F, split_words = True, frequency_threshold = 1):
    """
    given a document stored in a dictionary *tbd*, mapping an identifier key 
    onto a string containing the text, and a set of *all_verses*
    (the shared identifier keys between tbd and the source document(,
    this function returns a sparse matrix *fragments* of identifier key (rows) 
    to substrings of the text (column), with the matrix being True if the fragment
    occurs for that identifier key and False otherwise.
    as well as dictionaries for the identification
    of the rows and columns.
    (memory/computation efficient format, but a bit densely written)
    """
    wordcount = Counter((w for l in F for w in l))
    if split_words: 
        word_fragments = {w : set(get_fragments('^%s$' % unidecode(w).lower())) for w in wordcount.keys() }
    else: 
        word_fragments = {w : {unidecode(w).lower()} for w in wordcount.keys() }
    
    fragment_count = Counter((f for w,F in word_fragments.items() for f in F if f != '' for i in range(wordcount[w])))
    fragment_ixx = {f:i for i,(f,c) in enumerate(fragment_count.most_common()) if c >= frequency_threshold }
    #
    R,C = [], []
    for line_ix, line_f in enumerate(F):
        if len(line_f) == 0: continue
        line_frags = list(map(lambda f : fragment_ixx[f],
                              filter(lambda f : f in fragment_ixx,
                                     set.union(*map(lambda w : word_fragments[w], line_f)))))
        R.extend([line_ix]*len(line_frags))
        C.extend(line_frags)
    fragments = csr_matrix((np.ones(len(R)), (R,C)), dtype=bool, shape = (len(F),max(C)+1))
    return fragments, fragment_ixx

In [12]:
def create_alignments(doc, dataset, min_freq = 1, verbose=False):
    #
    E,F = zip(*[([w.split('/')[2] for w in l.strip('\n').split(' ||| ')[1].split() 
                  if w.count('/') >= 3 and w.split('/')[3] in good_pos and w.split('/')[2] not in excluded_lemmas],
                  l.strip('\n').split(' ||| ')[0].split())
                 for l in open('./generated/%s_bitexts/%s.spc' % (dataset,doc))])
    e_fragments, e_dic = get_all_fragments(E, split_words=False, frequency_threshold=1)
    e_counts = Counter([unidecode(e).lower() for l in E for e in l if e != ''])
    e_seed = {e : e_dic[e] for e in e_counts if e_counts[e] >= min_freq and e != ''}
    f_fragments, f_dic = get_all_fragments(F, split_words=True, frequency_threshold=min_freq)
    f_list = np.array(sorted(f_dic, key = lambda k : f_dic[k]))
    #
    # get TEs
    tes, te_words = {}, {}
    for erank,(e,ei) in enumerate(sorted(e_seed.items(), key = lambda x : -e_counts[x[0]])):
        pos = e_fragments[:,ei].nonzero()[0]
        tes[e] = extract_tes(pos, f_fragments, f_dic, coverage=.95, min_trans=0.01, min_backtrans=0.10)
        #tes[e] = merge_similar_tes(tes[e])
        te_words[e] = { te : get_te_words(f_fragments, f_list, te, te_pos) for te,te_pos in tes[e].items() }
        if verbose: print(doc, e, '%d/%d' % (erank,len(e_seed)), e_counts[e], datetime.now(), 
                          Counter({t:len(p) for t,p in tes[e].items()}).most_common(3))
    print(doc, datetime.now(), len(tes))
    with open('./generated/%s_output/%s.json' % (dataset,doc),'w') as fout:
        fout.write(json.dumps({'tes' : tes, 'te_words' : te_words}))

In [ ]:
def longest_nonoverlapping(m, rev, frags):
    frags = [rev[f] for f in frags]
    Ma = [mi for mi in frags if m in mi and 
          next((False for mj in frags if mj != mi and mi in mj),True)]
    return Ma

def get_te_words(fragments_F, rev, te, te_pos):
    ctr = Counter()
    for pos in te_pos:
        frags = fragments_F[pos].nonzero()[1]
        Ma = longest_nonoverlapping(te, rev, frags)
        for m in Ma:
            ctr[m] += 1
    return ctr

In [ ]:
def extract_tes(pos_ixx, fragments, fixx, coverage = 0.95, min_trans = 0.05, min_backtrans = 0.25, verbose=False):
    """
    implements one forward pass of the Liu et al. (2023) approach.
    Takes a list *pos* of positive instances (row identifiers for fragments)
    As well as the sparse matrix *fragments* and the two dictionaries
    (vixx -- verse_ixx and fixx -- fragment_ixx).
    The parameters determine the fragments considered in the extraction: the iteration keeps going until
    either no good fragments can be found (significance under .001) or the *coverage* has been reached.
    *min_trans* is a float [0,1] that filters out all fragments that occur in hit verses in less than min_trans
    proportion of all hit verses.
    """
    neg_ixx = list(set(range(fragments.shape[0]))-set(pos_ixx))
    flist = [None] * len(fixx)
    for k,v in fixx.items():
        flist[v] = k
    #sorted(fixx, key = lambda k : fixx[k])
    #
    pos_tot_orig = pos_tot = len(pos_ixx)
    neg_tot = len(neg_ixx)
    #
    pos_ct = fragments[pos_ixx].sum(0).A[0]
    neg_ct = fragments[neg_ixx].sum(0).A[0]
    #
    good_fragments = np.where((pos_ct >= 1) & ((pos_ct/pos_tot_orig) >= min_trans) &
                             (pos_ct/(pos_ct+neg_ct) >= min_backtrans))[0]
    string_props = [(f[0] == '^', f[-1] == '$', len(f)) for f in flist]
                    #len(re.sub('[.*]', '', f)))
    #
    hits = defaultdict(lambda : [])
    ct = 0
    fe_scores = {}
    while len(pos_ixx) >= (1-coverage) * pos_tot_orig:
        ct += 1
        #
        # GET BEST
        assoc_scores = Counter()
        for f in good_fragments:
            table = ((pos_ct[f],pos_tot-pos_ct[f]),(neg_ct[f],neg_tot-neg_ct[f]))
            try: fe_score = fe_scores[table]
            except KeyError: fe_score = fe_scores[table] = -np.log(fisher_exact(table, alternative='greater')[1])
            assoc_scores[f] = (fe_score, string_props[f])
        best, best_score = next((x for x in assoc_scores.most_common(1)),(None,None))
        if best == None or best_score[0] < -np.log(5e-2):
            break
        # print([flist[k] for k,v in assoc_scores.most_common(10)])
        #
        # UPDATE
        new_pos_ixx = []
        for pos_v in pos_ixx:
            if fragments[pos_v,best] > 0: hits[flist[best]].append(int(pos_v))
            else: new_pos_ixx.append(pos_v)
        #
        pos_ixx = new_pos_ixx
        pos_tot = len(pos_ixx)
        pos_ct = fragments[pos_ixx].sum(0).A[0]
        neg_ct = fragments[neg_ixx].sum(0).A[0]
        #
        good_fragments = np.where((pos_ct >= 1) & (pos_ct/pos_tot_orig >= min_trans) &
                                    (pos_ct/(pos_ct+neg_ct) >= min_backtrans))[0]
        if verbose: print(ct, flist[best])
    return hits


## execution

In [46]:
# DoReCo
with Pool(8) as p:
    p.starmap(create_alignments, map(lambda doc : (doc.strip('\n'), 'doreco', 1), open('./doreco_doculects.txt')))

/tmp/ipykernel_2603394/43898795.py:59: RuntimeWarning: invalid value encountered in divide
  (pos_ct/(pos_ct+neg_ct) >= min_backtrans))[0]
/tmp/ipykernel_2603394/43898795.py:59: RuntimeWarning: invalid value encountered in divide
  (pos_ct/(pos_ct+neg_ct) >= min_backtrans))[0]
/tmp/ipykernel_2603394/43898795.py:59: RuntimeWarning: invalid value encountered in divide
  (pos_ct/(pos_ct+neg_ct) >= min_backtrans))[0]
/tmp/ipykernel_2603394/43898795.py:59: RuntimeWarning: invalid value encountered in divide
  (pos_ct/(pos_ct+neg_ct) >= min_backtrans))[0]
/tmp/ipykernel_2603394/43898795.py:59: RuntimeWarning: invalid value encountered in divide
  (pos_ct/(pos_ct+neg_ct) >= min_backtrans))[0]
/tmp/ipykernel_2603394/43898795.py:59: RuntimeWarning: invalid value encountered in divide
  (pos_ct/(pos_ct+neg_ct) >= min_backtrans))[0]
/tmp/ipykernel_2603394/43898795.py:59: RuntimeWarning: invalid value encountered in divide
  (pos_ct/(pos_ct+neg_ct) >= min_backtrans))[0]
/tmp/ipykernel_2603394/4389

jeha1242 2025-04-18 17:14:39.731175 514
apah1238 2025-04-18 17:14:47.498563 670
goem1240 2025-04-18 17:14:52.368940 989
guri1247 2025-04-18 17:14:53.240213 724
cabe1245 2025-04-18 17:14:57.094581 813
jeju1234 2025-04-18 17:15:17.597729 963
hoch1243 2025-04-18 17:15:18.821600 655
anal1239 2025-04-18 17:15:28.313230 1189


/tmp/ipykernel_2603394/43898795.py:40: RuntimeWarning: divide by zero encountered in log
  except KeyError: fe_score = fe_scores[table] = -np.log(fisher_exact(table, alternative='greater')[1])


ligh1234 2025-04-18 17:15:29.614844 459
cash1254 2025-04-18 17:15:35.882616 976
arap1274 2025-04-18 17:15:37.842005 1056
kaka1265 2025-04-18 17:15:41.626992 1804
goro1270 2025-04-18 17:15:42.465707 1115


/tmp/ipykernel_2603394/43898795.py:40: RuntimeWarning: divide by zero encountered in log
  except KeyError: fe_score = fe_scores[table] = -np.log(fisher_exact(table, alternative='greater')[1])


ngal1292 2025-04-18 17:15:43.106620 491
kark1256 2025-04-18 17:15:53.483402 887
dolg1241 2025-04-18 17:15:54.064859 1389
nngg1234 2025-04-18 17:16:00.591341 1054
sanz1248 2025-04-18 17:16:07.302533 710
sout2856 2025-04-18 17:16:23.515610 941
savo1255 2025-04-18 17:16:27.032253 564
ruul1235 2025-04-18 17:16:46.806399 1297
pnar1238 2025-04-18 17:16:51.132689 1340
sumi1235 2025-04-18 17:16:55.871911 735
sadu1234 2025-04-18 17:17:01.953965 969
teop1238 2025-04-18 17:17:03.136264 667
even1259 2025-04-18 17:17:04.313404 1038
vera1241 2025-04-18 17:17:17.646562 771
texi1237 2025-04-18 17:17:22.043539 613
beja1238 2025-04-18 17:17:23.210239 1317
kama1351 2025-04-18 17:17:27.064510 1421
svan1243 2025-04-18 17:17:28.166817 1224
warl1254 2025-04-18 17:17:30.201208 442
komn1238 2025-04-18 17:17:38.227544 1503
taba1259 2025-04-18 17:17:38.712080 583
trin1278 2025-04-18 17:18:01.342957 1420
urum1249 2025-04-18 17:18:36.065223 976


In [ ]:
# OPUS
with Pool(4) as p:
    p.starmap(create_alignments, map(lambda doc : (doc.strip('\n'), 'opus', 30, True), ['es','fi', 'tr', 'el']))